# Primary Notebook

## Local Code Imports - Do not delete

In [1]:
# DO NOT REMOVE THESE
%load_ext autoreload
%autoreload 2

In [2]:
# DO NOT REMOVE This
%reload_ext autoreload

In [3]:
# Uncomment to filter warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
# DO NOT REMOVE
# import local src module -
from src import recapp as ra

# EDA, Data Cleaning & Feature Engineering

In [ ]:
user_factors = pd.read_csv('../data/processed/user_factors_unstacked.csv')

In [ ]:
user_factors_scaled = pd.read_csv('../data/processed/user_factors_scaled.csv')

In [ ]:
user_factors_scaled.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
centroid_ratings_T_df = cm.centroid_ratings(centroids, item_factors_unstacked)
centroid_ratings_T_df.head()

# Recommendations

In [5]:
ra.recommendations()

Enter a ranking from 1 (lowest) to 5 (highest) for the following movies. If you have not seen the movie, press enter.
Whiplash (2013): 
Can't Change the Meeting Place (1979): 
Princess Mononoke (Mononoke-hime) (1997): 
Wallace & Gromit: The Wrong Trousers (1993): 
Cinema Paradiso (Nuovo cinema Paradiso) (1989): 
M (1931): 
Silence of the Lambs, The (1991): 5
Third Man, The (1949): 2
All About Eve (1950): 
Chinatown (1974): 
Casablanca (1942): 5
Planet Earth II (2016): 
It Happened One Night (1934): 
The Blue Planet (2001): 
Goodfellas (1990): 3
Life (2009): 
Notorious (1946): 
To Kill a Mockingbird (1962): 4
Intouchables (2011): 
Paths of Glory (1957): 
The Adventures of Sherlock Holmes and Doctor Watson: 
Voices from the List (2004): 
Human Planet (2011): 
American Beauty (1999): 1
Children of Paradise (Les enfants du paradis) (1945): 
Life Is Beautiful (La Vita è bella) (1997): 
Operation 'Y' & Other Shurik's Adventures (1965): 
My Neighbor Totoro (Tonari no Totoro) (1988): 
Bicycle 